In [98]:
import pandas as pd
import wikipedia

In [84]:
names = pd.read_csv('data/firstnames.csv', delimiter=';')
female_names = names[names['Germany'].notnull() & names['gender'].str.contains('F')]['name']
male_names = names[names['Germany'].notnull() & names['gender'].str.contains('M')]['name']

In [85]:
df = pd.read_csv('data/berlin-streets.csv', delimiter='\t', header=0, names=['node', 'id', 'lat', 'long', 'country', 'city', 'street'])
df.drop_duplicates(subset=['street'], inplace=True)
df.head()

,node,id,lat,long,country,city,street
0,node,26735749,13.322832,52.506912,DE,Berlin,Knesebeckstraße
1,node,26735759,13.318092,52.506207,DE,Berlin,Kantstraße
2,node,26735763,13.320785,52.507339,DE,Berlin,Pestalozzistraße
3,node,26756830,13.328194,52.530825,DE,Berlin,Beusselstraße
4,node,26867411,13.294379,52.501851,DE,Berlin,Heilbronner Straße


In [110]:
df['woman'] = df.transform({'street': lambda x : any(y in x.split('-') for y in female_names)})
df['man'] = df.transform({'street': lambda x : any(y in x.split('-') for y in male_names)})
df[df['woman']]

,node,id,lat,long,country,city,street,woman,man
43,node,54854684,13.308889,52.570945,DE,Berlin,Auguste-Viktoria-Allee,True,False
77,node,69226092,13.372484,52.508197,DE,Berlin,Marlene-Dietrich-Platz,True,True
191,node,203469159,13.542597,52.526871,DE,Berlin,Helene-Weigel-Platz,True,False
281,node,256754256,13.411436,52.525898,DE,Berlin,Rosa-Luxemburg-Straße,True,False
286,node,257704934,13.211915,52.546258,DE,Berlin,Frieda-Arnheim-Promenade,True,False
587,node,268915217,13.378432,52.460038,DE,Berlin,Kaiserin-Augusta-Straße,True,False
607,node,268915292,13.289548,52.514133,DE,Berlin,Sophie-Charlotten-Straße,True,False
641,node,268915401,13.278708,52.459455,DE,Berlin,Königin-Luise-Straße,True,False
1011,node,269676264,13.366850,52.521282,DE,Berlin,Elisabeth-Abegg-Straße,True,False
1599,node,289665359,13.423728,52.560983,DE,Berlin,Jenny-Lind-Straße,True,False


In [109]:
women = df[df['woman']].apply(lambda x : ' '.join(x['street'].split('-')[:-1]), axis = 1)
# women['name'] = women.transform({'street' : lambda x : ' '.join(x.split('-')[:-1])})
women.head()

43     Auguste Viktoria
77     Marlene Dietrich
191       Helene Weigel
281      Rosa Luxemburg
286      Frieda Arnheim
dtype: object

In [108]:
wikipedia.set_lang('de')
print(women.iloc[0])
wikipedia.page(wikipedia.search(women.iloc[0])[0]).title

Auguste Viktoria


'Auguste Viktoria von Schleswig-Holstein-Sonderburg-Augustenburg'

In [124]:
results = women.apply(lambda x : wikipedia.search(x))

In [153]:
for idx, r in results[results.apply(lambda x : len(x)) < 10].iteritems():
    print({women.loc[idx]: r})

{'Hildegard Marcusson': ['Erwin Marcusson', 'Friedrichs-Waisenhaus Rummelsburg', 'Liste der Biografien/Mara–Marh', 'Liste der Straßen und Plätze in Berlin-Wilmersdorf', 'Nekrolog 1976', 'Liste der Straßen und Plätze in Berlin-Rummelsburg', 'Liste der Kulturdenkmale in Berlin-Rummelsburg']}
{'Amelie Beese': ['Amelie (Vorname)', 'Melli Beese', 'Beese', 'Liste bekannter Personen der Luftfahrt', 'Liste der Straßennamen von Frankfurt am Main/M', 'Garten der Frauen', 'Liste der Ehrengräber in Berlin', 'Liste der Straßen und Plätze in Berlin-Kladow']}
{'Rosa Reinglass': ['Liste der Straßen und Plätze in Berlin-Hakenfelde', 'Liste der Stolpersteine in Berlin-Wilhelmstadt']}
{'Anna Bruseberg': ['Liste der Straßen und Plätze in Berlin-Französisch Buchholz']}
{'Hiltrud Dudek': ['Südpark (Berlin-Rudow)', 'Liste der Straßen und Plätze in Berlin-Rudow', 'Liste der Kinos im Berliner Bezirk Neukölln']}
{'Gertrud Dorka': ['Gertrud Dorka', 'Dorka', 'Landesgeschichtliche Vereinigung für die Mark Brandenb

In [149]:
women.loc[230352]

'Anna Bruseberg'